# 反爬：登入授權模擬

* 了解「登入權限機制」的反爬蟲機制
* 「登入權限機制」反爬蟲的因應策略

## 作業目標

* 找一個需要登入的網站試試看，並說明思考流程
（如果不知道要用哪個網站的話，可以試試看 https://github.com/new 網址，未登入時會被導向登入頁）



### https://github.com/login 

* GitHub登入的URL雖然時https://github.com/login，但發生了302重定向，其真正提交POST表單資料的URL是 https://github.com/session ，當登入成功時，則會跳轉到 https://github.com/ 首頁。
登入方式分析：

第一，用 form 表單方式提交資訊，

第二，有 csrf_token,

第三 ，是以post請求傳送使用者名稱和密碼時，需要第一次get請求的cookie

第四，登入成功以後，請求其他頁面是隻需要帶第一次登入成功以後返回的cookie就可以。

tips:先以 get 傳送的請求獲取我們想要的 token 和 cookie

In [1]:
import requests
from bs4 import BeautifulSoup
r1 = requests.get('https://github.com/login')
soup = BeautifulSoup(r1.text,features='lxml') #生成soup 物件
s1 = soup.find(name='input',attrs={'name':'authenticity_token'}).get('value') #查到我們要的token
r1_cookies = r1.cookies.get_dict() # 下次提交使用者名稱時用的cookie
print(r1_cookies)
print(s1)

{'_octo': 'GH1.1.1909232386.1579152147', 'logged_in': 'no', '_gh_sess': 'bWt0L1RPQUk3ZjlCd2w3d0daQlZ6Y1lVN0V1cDgrUjMwNUMwQU81MVpJNzVPTVUxakhVaERpN2FLMVUzVDlqNy9MSU0raW82THA0YWRWUHgvbm5hcG05d0FubHNLQkhwUzQ1MXp3QXdHWFg4dUNqUTNCdHZSek9CWXpoTFZrbkxnWG9KK0hTQUxGdGpkWWNXSi9LMkp4NmxWZnM4MitKM0liMHFNRzBTenRIWER2cnlZRlVTOURxTWpmcEZ2dEZ0Rnc2SGQ5WDAvMnV1VGdPcXIvdksydVlCTDg2SUw3M1BMMlQ3bnNWQUp5bmxieWRqS3FLSnNNeE80THNSaVBDcC0tdzNQZnlHUi9scjdUOVJ0aXR6b0NKUT09--fa1b07268edb6450d64b5d244a263cc4b53164d1', 'has_recent_activity': '1'}
+AJ2fwByXZph3aoZwlGCk7YWkmouo3IkRj1vMGKJBQgU7SWMG94AozlXakpDPiNC7NKYE2MCr2Z1ORj5dzSWrA==


### 使用post請求傳送使用者名稱和密碼時，需要第一次get請求的cookie
* https://github.com/session 登入

In [3]:
r2 = requests.post(
  'https://github.com/session',
  data ={
    'commit':'Sign in',
    'utf8':'✓',
    'authenticity_token':s1, # 這裡需要第一次的token
    'login':'kurt.ni@gmail.com',
    'password':'PASSWORD' # 填入正確密碼
  },
  cookies = r1.cookies.get_dict(), # 這裡需要第一次的cookie
)
print(r2.cookies.get_dict())

{'logged_in': 'yes', '__Host-user_session_same_site': 'rshgi6sMZNuAV24F-N8Xb-5t3Lc_C77TfAf541tSbVEmPLg9', '_gh_sess': 'OGNSbzlENnl2czMvV3Exd1FHUFZhT3QyNnFSclh5enBCQUtGd3hmSnVaKzV0UUhEQ2xDRVZPUkUvSzBwQldXcEk0QkhNL2ZjYm9XbGpmU1VCWFJKZC9EdnhwaE5jb1VTRFo0THZNL3lERTVaQnE3amVmUzhoeDRPY2hHYkZ6aFlnejBUWlkvNmtneU12d0NBOE5RektlSGdTMEtWQk9aUHl3eENzaCs3c1VjPS0tdDR1RG5pcUlOZGhPSnVCekhMRVFhZz09--202f68fb68ef96622f59f1cd32ea36ef26c2ba84', 'has_recent_activity': '1', 'user_session': 'rshgi6sMZNuAV24F-N8Xb-5t3Lc_C77TfAf541tSbVEmPLg9'}


In [4]:
r3 = requests.get(
    'https://github.com/kurtni',  #檢視個人的詳情頁
    cookies = r2.cookies.get_dict())
print(r3.text)



<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">
  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars3.githubusercontent.com">
  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">
  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">



  <link crossorigin="anonymous" media="all" integrity="sha512-hddDYPWR0gBbqLRmIZP242WMEiYsVkYI2UCYCVUHB4h5DhD2cbtFJYG+HPh21dZGb+sbgDHxQBNJCBq7YbmlBQ==" rel="stylesheet" href="https://github.githubassets.com/assets/frameworks-02a3eaa24db2bd1ed9b64450595fc2cf.css" />
  
    <link crossorigin="anonymous" media="all" integrity="sha512-8ArQLJDpV7H9drCFVkdRXJ58nXFBDZckzZG8zyc6aEx7HrrJvInon7ME8lx9yTCprau6mu4fqQFnGym